In [58]:
%run './remodelisations/communes.ipynb'
%run './remodelisations/dep.ipynb'
%run './remodelisations/regions.ipynb'
# %run './remodelisations/statistiques.ipynb' # les stats ont des problèmes avec les fk -> ref vers des code de communes inconnues


In [59]:
import psycopg2 as postgres

In [60]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()


In [61]:
tables = ['departements_cheflieu','regions_cheflieu', 'communes', 'departements', 'regions','indicateurs','statistiques']

In [62]:
# A enlever avant rendu 

drop_tables = ''
for table in tables:
    drop_tables += f'DROP TABLE IF EXISTS {table} CASCADE;'


In [63]:
cur.execute(drop_tables)
conn.commit()

In [64]:
cur.execute(open("sql/create.sql", "r").read())
conn.commit()

In [65]:
files = [
        'regions', 
        'departements',
        'communes',
        'departements_cheflieu',
        'regions_cheflieu',
        # 'statistiques'
    ]

for file in files:
    path = f'./csv/filtered/{file}.csv'
    cur.copy_from(open(path, 'r'), file, sep=',')
    conn.commit()


BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY communes, line 1: "01001,84,01,ABERGEMENT CLEMENCIAT,L'Abergement-Clémenciat"


In [ ]:
conn.close()